In [1]:
!pip install emoji

In [2]:
!pip install nltk

In [3]:
!pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.7 MB/s eta 0:00:0000:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [4]:
!pip install jieba

In [5]:
!pip install indoNLP

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.7 MB/s eta 0:00:00


In [6]:
!pip install easygoogletranslate

In [7]:
import pandas as pd
import re
import nltk
import jieba
import string
import emoji
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from indoNLP.preprocessing import replace_slang
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from snowballstemmer import stemmer
from easygoogletranslate import EasyGoogleTranslate

import torch
import torch.nn as nn
import warnings

warnings.filterwarnings("ignore", message="Some weights of BertForSequenceClassification were not initialized from the model checkpoint")

In [8]:
# abbreviation list taken from https://www.kaggle.com/code/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing#Slangs
# English Abbreviation
abbreviations = {
    "$": " dollar ",
    "€": " euro ",
    "4ao": "for adults only",
    "a.m": "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "pls" : "please",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [9]:
class EnTextCleaner:
    def __init__(self, text, abbreviations):
        self.dataframe = pd.DataFrame(data={'text':[text]})
        self.abbreviations = abbreviations

    def lower_casing(self):
        """Convert all text to lowercase."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe["text"].str.lower()
        return self

    def remove_urls(self):
        """Remove urls."""
        url_re = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe["cleaned_text"].apply(lambda text: url_re.sub('', text))
        return self

    def remove_usernames(self):
        """Remove usernames that start with '@'."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('@[^\s]+', '', text))
        return self

    def remove_html(self):
        """Remove HTML tags."""
        tag_re = re.compile(r'<[^>]+>')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: tag_re.sub('', text))
        return self

    def remove_hex_and_unicode(self):  #found in indonesian
        """Remove hexadecimal and Unicode escape sequences."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(
            lambda text: re.sub(r'\\x[0-9a-fA-F]+|\\u[0-9a-fA-F]{4}', '', text))
        return self

    def remove_emoji(self):
        """Remove emoji from text"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: emoji.replace_emoji(text))
        return self

    def remove_punctuation(self):
        """Remove punctuation from the text, including Chinese punctuation if specified."""
        en_punct_pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: en_punct_pattern.sub('', text))
        return self

    def remove_numbers(self):
        """Remove numbers"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('\d+', '', text))
        return self

    def remove_whitespace(self):
        """Remove whitespaces"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: " ".join(text.split()))
        return self

    def chat_word_conversion(self):
        """Convert chat words into formal words"""
        def convert_abbrev(text):
            #tokenize the text first
            tokens = word_tokenize(text)
            converted_tokens = [self.abbreviations.get(token.lower(), token) for token in tokens]
            return ' ' .join(converted_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(convert_abbrev)
        return self

    def remove_en_stopwords(self):
        """Remove stopwords"""
        def remove_stopwords(text):
            tokens = word_tokenize(text)
            english_stopwords = set(stopwords.words('english')) | {"retweet", "user"}
            removed_tokens = [token for token in tokens if token not in english_stopwords]
            return ' ' .join(removed_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(remove_stopwords)
        return self

    def en_lemmatizer(self):
        """Lemmatize English"""

        def lemmatize_words(text):
            """Function to lemmatize words in a list of tokens"""
            lemmatizer = WordNetLemmatizer()
            wordnet_map = {
                'N': wordnet.NOUN,
                'V': wordnet.VERB,
                'R': wordnet.ADV,
                'J': wordnet.ADJ
            }

            tokens = word_tokenize(text)

            # Perform part-of-speech tagging
            pos_tagged_tokens = nltk.pos_tag(tokens)
            # Lemmatize each word based on its part-of-speech
            lemmatized_tokens = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_tokens]
            return ' ' .join(lemmatized_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lemmatize_words)
        return self

    def get_cleaned_dataframe(self):
        """Return the cleaned DataFrame."""
        return self.dataframe

In [10]:
# take from online and combine chatgpt generated stopwords
# https://www.ranks.nl/stopwords/chinese-stopwords
zh_stopwords = ['的','一','不','在','人','有','是','为','以','于','上','他','而','后','之','来','及','了',
                '因','下','可','到','由','这','与','也','此','但','并','个','其','已','无','小','我','们',
                '起','最','再','今','去','好','只','又','或','很','亦','某','把','那','你','乃','它','吧',
                '被','比','别','趁','当','从','得','打','凡','儿','尔','该','各','给','跟','和','何','还',
                '即','几','既','看','据','距','靠','啦','另','么','每','嘛','拿','哪','您','凭','且','却',
                '让','仍','啥','如','若','使','谁','虽','随','同','所','她','哇','嗡','往','些','向','沿',
                '哟','用','咱','则','怎','曾','至','致','着','诸','自','呢','吗','啊','哦','呀','噢','哈',
                '呃','嗯','唉','啦','呗','嘿','哒','啵','喽','嘛','咧','咋','哩','呼','吱','呜','咦','咚',
                '哐','咔','哧','咕','呶','噻','哇','哔','嗖','嘎','嗒','嘘','嘁','呕','啪','啷','啪','喳',
                '嘤','轰','哼','唿','嚯','呵','嚓','哒','嗡','嘻','嘟','嗑','嗬','嗔','嗦','嗝','嗄','嗯', 
                '嗨','喽','嘿','呀','呦','矣','哉','俺','尔','耶','呗','咻','咿','哎','哏','哗','咯','啰', 
                '啧','喏','喔','嗷','嘈','嘤','嗉','呷','呱','呤','噼','啪','叽','咣','咭','哒','嗒','嗤',
                '哙','哚','哜','嗖','嗑','嗲','嘚','嗌','嘧','嘭','哓','嗵','都']

In [11]:
class ZhTextCleaner:
    def __init__(self, text, zh_stopwords):
        self.dataframe = pd.DataFrame(data={'text':[text]})
        self.zh_stopwords = zh_stopwords

    def remove_urls(self):
        """Remove urls."""
        url_re = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe["text"].apply(lambda text: url_re.sub('', text))
        return self

    def remove_usernames(self):
        """Remove usernames that start with '@'."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('@[^\s]+', '', text))
        return self

    def remove_html(self):
        """Remove HTML tags."""
        tag_re = re.compile(r'<[^>]+>')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: tag_re.sub('', text))
        return self

    def remove_hex_and_unicode(self):  #found in indonesian
        """Remove hexadecimal and Unicode escape sequences."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(
            lambda text: re.sub(r'\\x[0-9a-fA-F]+|\\u[0-9a-fA-F]{4}', '', text))
        return self

    def remove_emoji(self):
        """Remove emoji from text"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: emoji.replace_emoji(text))
        return self

    def remove_punctuation(self):
        """Remove punctuation from the text, including Chinese punctuation if specified."""
        zh_punct_pattern = re.compile('[' + re.escape(string.punctuation) + "\u3000-\u303F" +"\uFF00-\uFFEF" +"\u2000-\u206F" +"\u2E00-\u2E7F" + ']+')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: zh_punct_pattern.sub('', text))
        return self

    def remove_numbers(self):
        """Remove numbers"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('\d+', '', text))
        return self

    def remove_whitespace(self):
        """Remove whitespaces"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: " ".join(text.split()))
        return self

    def remove_zh_stopwords(self):
        """Remove stopwords"""
        def remove_stopwords(text):
            tokens = jieba.lcut(text)
            removed_tokens = [token for token in tokens if token not in self.zh_stopwords]
            return ' ' .join(removed_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(remove_stopwords)
        return self

    def get_cleaned_dataframe(self):
        """Return the cleaned DataFrame."""
        return self.dataframe

In [12]:
class IdTextCleaner:
    def __init__(self, text, abbreviations):
        self.dataframe = pd.DataFrame(data={'text':[text]})
        self.abbreviations = abbreviations

    def lower_casing(self):
        """Convert all text to lowercase."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe["text"].str.lower()
        return self

    def remove_urls(self):
        """Remove urls."""
        url_re = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe["cleaned_text"].apply(lambda text: url_re.sub('', text))
        return self

    def remove_usernames(self):
        """Remove usernames that start with '@'."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('@[^\s]+', '', text))
        return self

    def remove_html(self):
        """Remove HTML tags."""
        tag_re = re.compile(r'<[^>]+>')
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: tag_re.sub('', text))
        return self

    def remove_hex_and_unicode(self):  #found in indonesian
        """Remove hexadecimal and Unicode escape sequences."""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(
            lambda text: re.sub(r'\\x[0-9a-fA-F]+|\\u[0-9a-fA-F]{4}|[\u2600-\u26FF]', '', text))
        return self

    def remove_emoji(self):
        """Remove emoji from text"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: emoji.replace_emoji(text))
        return self

    def remove_punctuation(self):
        """Remove punctuation from the text, including Chinese punctuation if specified."""
        en_punct_pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: en_punct_pattern.sub('', text))
        return self

    def remove_numbers(self):
        """Remove numbers"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: re.sub('\d+', '', text))
        return self

    def remove_whitespace(self):
        """Remove whitespaces"""
        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(lambda text: " ".join(text.split()))
        return self

    def chat_word_conversion(self):
        """Convert chat words into formal words"""
        def convert_abbrev(text):
            tokens = word_tokenize(text)
            converted_tokens = [self.abbreviations.get(token.lower(), token) for token in tokens]
            return ' ' .join(converted_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(convert_abbrev)
        return self

    def replace_slang(self):
        """Convert slang into formal words"""
        def replace_indonesian_slang(text):
            text_with_slang_replaced = replace_slang(text)
            return text_with_slang_replaced

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(replace_indonesian_slang)
        return self

    def remove_id_stopwords(self):
        """Remove stopwords"""
        def remove_stopwords(text):
            tokens = word_tokenize(text)
            indonesian_stopwords = set(stopwords.words('indonesian')) | {"retweet", "user"}
            removed_tokens = [token for token in tokens if token not in indonesian_stopwords]
            return ' ' .join(removed_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(remove_stopwords)
        return self

    def id_stemmer(self):
        """Stem indonesian text"""
        def stem_indonesian(text):
            tokens = word_tokenize(text)
            stemmer_id = stemmer("indonesian")
            stemmed_tokens = [stemmer_id.stemWord(token) for token in tokens]
            return ' ' .join(stemmed_tokens)

        self.dataframe.loc[:, 'cleaned_text'] = self.dataframe['cleaned_text'].apply(stem_indonesian)
        return self

    def get_cleaned_dataframe(self):
        """Return the cleaned DataFrame."""
        return self.dataframe

In [13]:
def preprocessing(text, language, abbreviations, zh_stopwords):
    if language=='en':
        #peform preprocessing for each language
        en_cleaner = EnTextCleaner(text, abbreviations)
        en_cleaner.lower_casing()
        en_cleaner.remove_urls()
        en_cleaner.remove_usernames()
        en_cleaner.remove_html()
        en_cleaner.remove_hex_and_unicode()
        en_cleaner.remove_emoji()
        en_cleaner.remove_punctuation()
        en_cleaner.remove_numbers()
        en_cleaner.remove_whitespace()
        en_cleaner.chat_word_conversion()
        en_cleaner.remove_en_stopwords()
        en_cleaner.en_lemmatizer()
        en_cleaned = en_cleaner.get_cleaned_dataframe()
        return en_cleaned.iloc[0]['cleaned_text']
    
    elif language=='zh':
        zh_cleaner = ZhTextCleaner(text, zh_stopwords)
        zh_cleaner.remove_urls()
        zh_cleaner.remove_usernames()
        zh_cleaner.remove_html()
        zh_cleaner.remove_hex_and_unicode()
        zh_cleaner.remove_emoji()
        zh_cleaner.remove_punctuation()
        zh_cleaner.remove_numbers()
        zh_cleaner.remove_whitespace()
        zh_cleaner.remove_zh_stopwords()
        zh_cleaned = zh_cleaner.get_cleaned_dataframe()
        return zh_cleaned.iloc[0]['cleaned_text']
    
    elif language=='id':
        id_cleaner = IdTextCleaner(text, abbreviations)
        id_cleaner.lower_casing()
        id_cleaner.remove_urls()
        id_cleaner.remove_usernames()
        id_cleaner.remove_html()
        id_cleaner.remove_hex_and_unicode()
        id_cleaner.remove_emoji()
        id_cleaner.remove_punctuation()
        id_cleaner.remove_numbers()
        id_cleaner.remove_whitespace()
        id_cleaner.chat_word_conversion()
        id_cleaner.replace_slang()
        id_cleaner.remove_id_stopwords()
        id_cleaner.id_stemmer()
        id_cleaned = id_cleaner.get_cleaned_dataframe()
        return id_cleaned.iloc[0]['cleaned_text']
    

In [14]:
# class SimpleBiLSTM(nn.Module):
#     def __init__(self, embedding_dim, hidden_dim,num_layers,batch_first,bidirectional):
#         super(SimpleBiLSTM, self).__init__()
#         # Define a bidirectional LSTM
#         self.lstm = nn.LSTM(input_size=embedding_dim, 
#                             hidden_size=hidden_dim, 
#                             num_layers=num_layers,
#                             batch_first=batch_first, 
#                             bidirectional=bidirectional)
        
#         # Define a fully connected layer to map the concatenated hidden states to a single output
#         self.fc = nn.Linear(in_features=2 * hidden_dim, out_features=1)

#     def forward(self, x):
#         # Forward pass through LSTM layer
#         _, (hn, _) = self.lstm(x)  # hn contains the hidden state for each direction of each layer
        
#         # Concatenate the last hidden states of the forward and backward passes from the last layer
#         hn_cat = torch.cat((hn[-2], hn[-1]), dim=1)

#         # Pass the concatenated hidden states through the fully connected layer and apply sigmoid activation
#         out = self.fc(hn_cat)
#         out = torch.sigmoid(out)

#         # Squeeze the output to remove the singleton dimension to match expected output shape (batch_size,)
#         return out.squeeze(1)
    

In [15]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mbert-models/pytorch/mbert/1/mBERT_over.pth


In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
mBERT_model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=2)
mBERT_model.load_state_dict(torch.load('/kaggle/input/mbert-models/pytorch/mbert/1/mBERT_over.pth', map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [17]:
def predict_class_mBert(input_text, language, abbreviations, zh_stopwords):
    # Assuming preprocessing is defined elsewhere
    input_data = preprocessing(input_text, language, abbreviations, zh_stopwords)
    input_ids = []
    attention_masks = []

    # Tokenize all texts
    encoded = tokenizer.encode_plus(
        input_data,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    with torch.no_grad():
        outputs = mBERT_model(input_ids, attention_mask=attention_masks)
    
    logits = outputs.logits
    probs = torch.softmax(logits, dim=1)
    predictions = torch.argmax(logits, dim=1)

    results = []
    for i, prob in enumerate(probs):
        pred_label = predictions[i].item()
        prob_of_pred_class = prob[pred_label].item()
        classification = "Non-Hate Speech" if pred_label == 1 else "Hate Speech"
        results.append((prob_of_pred_class, classification))
    
    return results[0]


In [18]:
# Display
text = "I hate you"
result = predict_class_mBert(text, "en", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Analysing... I hate you
Probability of being 'Hate Speech' is 79.10%
Final Verdict: Hate Speech


In [19]:
# Display
text = "I love you so much!!"
result = predict_class_mBert(text, "en", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Analysing... I love you so much!!
Probability of being 'Non-Hate Speech' is 58.37%
Final Verdict: Non-Hate Speech


In [20]:
# Display
text = "你去死吧"
result = predict_class_mBert(text, "zh", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.381 seconds.
Prefix dict has been built successfully.


Analysing... 你去死吧
Probability of being 'Hate Speech' is 82.00%
Final Verdict: Hate Speech


In [21]:
# Display
text = "你爱我呀"
result = predict_class_mBert(text, "zh", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Analysing... 你爱我呀
Probability of being 'Non-Hate Speech' is 61.85%
Final Verdict: Non-Hate Speech


In [22]:
# Display
text = "Kamu pergi mati"
result = predict_class_mBert(text, "id", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Analysing... Kamu pergi mati
Probability of being 'Hate Speech' is 63.51%
Final Verdict: Hate Speech


In [23]:
# Display
text = "Sekarang saya tidur"
result = predict_class_mBert(text, "id", abbreviations, zh_stopwords)

print("Analysing...", text)
print(f"Probability of being '{result[1]}' is {result[0] * 100:.2f}%")
print("Final Verdict:", result[1])

Analysing... Sekarang saya tidur
Probability of being 'Non-Hate Speech' is 88.87%
Final Verdict: Non-Hate Speech
